In [96]:
import pandas as pd
import sqlite3
from config import cmconf
from pathlib import Path
import numpy as np

# 데이터베이스 불러오기

In [97]:
dbpath = Path(cmconf['path']['root']) / Path(cmconf['path']['db'])
conn = sqlite3.connect(dbpath)
cur = conn.cursor()

# 기업과 관련된 재무제표 불러오기

In [98]:
corp_name = '삼성전자'

In [106]:
sql = '''
select c.name as 'corp_name', m.name as 'market', r.year, r.quarter, rt.name, r.date, ri.account_code, ri.account_name, ri.amount
from Report as r
inner join ReportItem as ri on ri.report = r.id
inner join StockCode as sc on sc.corporation = r.corporation
inner join corporation as c on c.id = r.corporation
inner join market as m on sc.market = m.id
inner join sector as st on c.sector = st.id
inner join ReportType as rt on rt.id = r.report_type
where
    c.name = ? and
    rt.name like '%포괄손익%연결%'
order by date;
'''
# columns = [
#     '회사명', '시장', '연도', '분기', '보고서유형', '결산일', '계정코드', '계정명', '금액'
# ]
df = pd.read_sql(sql, conn, params=(corp_name,))

In [107]:
pd.options.display.float_format = '{:,.0f}'.format
df

,corp_name,market,year,quarter,name,date,account_code,account_name,amount
0,삼성전자,유가증권시장상장법인,2017,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,ifrs_ProfitLoss,당기순이익(손실),"7,684,354,000,000"
1,삼성전자,유가증권시장상장법인,2017,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,ifrs_OtherComprehensiveIncome,기타포괄손익,"-4,932,049,000,000"
2,삼성전자,유가증권시장상장법인,2017,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,dart_OtherComprehensiveIncomeThatWillNotBeRecl...,후속적으로 당기손익으로 재분류되지 않는 포괄손익,"-39,214,000,000"
3,삼성전자,유가증권시장상장법인,2017,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,entity00126380_udf_IS_201732919385733_OtherCom...,순확정급여부채 재측정요소,"-39,098,000,000"
4,삼성전자,유가증권시장상장법인,2017,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,entity00126380_udf_IS_20164261642947_OtherComp...,관계기업 및 공동기업의 기타포괄손익에 대한 지분,"-116,000,000"
...,...,...,...,...,...,...,...,...,...
308,삼성전자,유가증권시장상장법인,2023,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_GainsLossesOnCashFlowHedgesNetOfTax,현금흐름위험회피파생상품평가손익,"8,108,000,000"
309,삼성전자,유가증권시장상장법인,2023,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_ComprehensiveIncome,총포괄손익,"7,554,100,000,000"
310,삼성전자,유가증권시장상장법인,2023,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_ComprehensiveIncomeAttributableToAbs...,포괄손익의 귀속,NaN
311,삼성전자,유가증권시장상장법인,2023,1분기보고서,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_ComprehensiveIncomeAttributableToOwn...,지배기업 소유주지분,"7,286,763,000,000"


## nan row 제거

In [108]:
## nan row 제거
df = df.dropna(subset=['amount'])
df.reset_index(drop=True)

## 의미있는 문자열로 변환
df.loc[:, 'quarter'] = df.loc[:, 'quarter'].str.replace('1분기보고서', '1Q')
df.loc[:, 'quarter'] = df.loc[:, 'quarter'].str.replace('반기보고서', '2Q')
df.loc[:, 'quarter'] = df.loc[:, 'quarter'].str.replace('3분기보고서', '2Q')
df.loc[:, 'quarter'] = df.loc[:, 'quarter'].str.replace('사업보고서', '4Q')

## 금액 변환(억 단위)
df.loc[:, 'amount'] = df['amount'].apply(lambda x: int(x) / 100000000)

df

,corp_name,market,year,quarter,name,date,account_code,account_name,amount
0,삼성전자,유가증권시장상장법인,2017,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,ifrs_ProfitLoss,당기순이익(손실),"76,844"
1,삼성전자,유가증권시장상장법인,2017,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,ifrs_OtherComprehensiveIncome,기타포괄손익,"-49,320"
2,삼성전자,유가증권시장상장법인,2017,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,dart_OtherComprehensiveIncomeThatWillNotBeRecl...,후속적으로 당기손익으로 재분류되지 않는 포괄손익,-392
3,삼성전자,유가증권시장상장법인,2017,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,entity00126380_udf_IS_201732919385733_OtherCom...,순확정급여부채 재측정요소,-391
4,삼성전자,유가증권시장상장법인,2017,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2017-03-31,entity00126380_udf_IS_20164261642947_OtherComp...,관계기업 및 공동기업의 기타포괄손익에 대한 지분,-1
...,...,...,...,...,...,...,...,...,...
307,삼성전자,유가증권시장상장법인,2023,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업장환산외환차이,"50,569"
308,삼성전자,유가증권시장상장법인,2023,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_GainsLossesOnCashFlowHedgesNetOfTax,현금흐름위험회피파생상품평가손익,81
309,삼성전자,유가증권시장상장법인,2023,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_ComprehensiveIncome,총포괄손익,"75,541"
311,삼성전자,유가증권시장상장법인,2023,1Q,포괄손익계산서(세후기타포괄손익) - 연결재무제표,2023-03-31,ifrs-full_ComprehensiveIncomeAttributableToOwn...,지배기업 소유주지분,"72,868"


## 자산대비 부채비율

In [109]:
df2 = df[df['account_code'].isin(['ifrs_Assets', 'ifrs_Equity', 'ifrs_Liabilities'])]
100 * df2.loc[df2['account_code'] == 'ifrs_Liabilities', 'amount'] / df2.loc[df2['account_code'] == 'ifrs_Assets', 'amount'].values

Series([], Name: amount, dtype: float64)

# 재무제표에서 해야할 질문
## 매출, 영업이익, 비용, 부채

   기업가치 = f(매출, 영업이익, 비용, 부채)

### 매출이 기간동안 증가하고 있는가?, 얼만큼 증가하는가?

### 영업이익이 기간동안 증가하고 있는가?, 얼만큼 증가하는가?

### 비용이 기간동안 감소하거나 정체하고 있는가?

### 부채가 기간동안 감소하거나 정체하고 있는가?

## 최근 4분기(1년) 동안 매출 증가 비율

# Macro 분석 지표
## 기준 금리
### 국채 시장 금리
### 회사채 시장 금리
## 경제성장률